# Submodule 2 – Determining Protein Structure using Phenix


## Overview
Once the reflections from a diffraction experiment have been scaled, the next step is to use them to solve the structure. The .hkl file from SCALEPAK will be used for this purpose. The method of molecular replacement is used if there is an existing structure that is similar to the one you wish to solve. In the event there is not a model structure, it is necessary to use anomalous diffraction methods like Multi or Single Anomalous Diffraction which was covered in Submodule 1. Many programs have been written over the years to handle this task, but it is most convenient to have the software needed for this bundled into a suite of programs. In the past the Crystallography and NMR system (CNS) and Collaborative Computational Project No. 4 (CCP4) were valuable resources for protein structure solution. Currently, one of the most user-friendly software suites is Phenix. To quote the authors, “Phenix is a comprehensive software package for macromolecular structure determination that handles data from any of these techniques.” 

## Learning Objectives
* Be able to solve a structure using SAD/MAD phasing in Phenix 
* Be able to solve a structure using native data for phasing with molecular replacement in Phenix

## Prerequisites
- Scaled diffraction data
- Relevant fasta file
- Confirm you can run Phenix software using our [Phenix GUI Notebook](submodule_0_Phenix.ipynb)
------------

## Evaluating Diffraction Data
After x-ray data are collected, we can evaluate the quality of the x-ray data. There are several parameters to consider which are defined below. 

Resolution: The smallest distance between crystal lattice planes that is resolved in the diffraction pattern1. What we see in the electron density is the lower the resolution the more closely the electron density maps match the structure model built. For example, at high resolution (<2.0 Å) maps look shrink-wrapped around the model and nearly all side chains are easily interpretable. At lower resolution (3.5-5.0 Å) It becomes difficult to discern between all but the largest residues. Helices look like tubes and strands meld together to look like sheets. However, the backbone trace of the polypeptide chain can still be seen clearly. 

R-factor: Average of mean difference between observations.

$R_{sym}$: R-factor for symmetry related reflections. Rsym is calculated by looking at the intensity of a reflection and comparing it to the average intensity of symmetry related reflections. This process is done for all reflections in the dataset. Consider a reflection hi that is repeated four times in the data. Ih,i is the intensity of the i-th measurement of that reflection. Therefore, there would the Ih,1, Ih,2, Ih,3, and Ih,4. Using the equation below, if there were four reflections with intensities of 4.1, 4.2, 4.8, and 5.0 would give an Rsym value of 8.3%. 

$$
R_{sym} = \frac{\sum\sum|I_{h,i}-<I_{h}>|}{\sum\sum I_{h,i}}|
$$

The lower the $R_{sym}$ value the better the date.

I/σ(I): Often called I over sigma, this is the signal to noise ratio of the data.

$Completeness^{2}$: The percentage of possible reflections that have been captured. Experimentally this is done by rotating the crystal in an x-ray beam to collect x-ray diffraction patterns. The rotation range in degrees needed for a complete dataset depends on the symmetry of the unit cell. For example, there are 11 point groups possible for protein crystals. A primitive, triclinic unit cell requires at least 180° rotation range, while a high symmetry cubic crystal can achieve a complete dataset with a range of just 35°. If the data are anomalous, each x-ray pattern can be collected followed by the next collected pattern at 180° away. This is to capture Friedel mates necessary to measure the anomalous signal, so a primitive triclinic crystal would need 360° to be complete. In practice, it is often best to collect as many frames as possible, since it is difficult to predict the exact number of frames necessary for complete data, and there are often blind regions in the crystal where reflections cannot be captured. 

Redundancy: This is the number of times the same indexed reflections were measured within a dataset.

Mosaicity: A measure of the spread in size and orientation of unit cells within a crystal. Some amount of mosaicity is unavoidable. However, too much can cause reflections to overlap which can be seen in broadening of reflections in the x-ray pattern. Mosaicity can result from disorder as a result of crystal growth and is introduced when freezing crystals in liquid nitrogen or butane before x-ray data collection at cryogenic temperatures. 

One final consideration is that data statistics are broken up into resolution shells. Unsurprisingly, values like I/σ(I), $R_{sym}$, completeness, and redundancy get worse at higher resolution. This is in part because the intensity of the reflections fall off at higher diffraction angles. Often data collection statistics list an additional value in parentheses to show the corresponding parameter in the highest resolution shell. In the chart below the values of redundance, completeness, $R_{sym}$ and I/σ(I) are listed with another value in parentheses next to them. The numbers in parentheses are the respective values at the highest resolution shell, which for this example extends to 1.82Å. The actual shells are dived up during scaling, but at high resolution typically only span a few fractions of an Angstrom. 

| | |
|--|--|
|Resolution(Å) |1.82 |
|Redundancy |4.4(3.1) |
|Completeness |98.5(94.0) |
|$R_{sym}$(%) |4.5(25.6) |
|I/σ(I) |22.7(3.3) |

<details>
    <summary>Click to Reveal Helpful Resources</summary>
    <li><a href=https://proteopedia.org/wiki/index.php/Resolution target="_blank">Proteopedia - Resolution</a></li>
    Also, an excellent resource that was used to write these definitions can be found here: <li><a href=https://pubmed.ncbi.nlm.nih.gov/10531520/ target="_blank">Dauter (1999)</a></li>
    </details>

--------
## Activity 1 - Solving a structure Using SAD/MAD Phasing
In this activity we will define the download_pdb function to pull .cif and .fasta files from the Protein Database (PDB) and use the downloaded files to begin solving protein structures.

#### We will use the following PDB IDs:
PDBID1: `1CRN` <br>
PDBID2: `2A3N`

#### Step 1 - Define Download Function

In [3]:
# Function to pull fasta and cif files from pdb based on pdbid
import os 
import requests

def download_pdb(pdbid):
    download_dir = f'Data/pdb_{pdbid}'
    os.makedirs(download_dir)
    
    # Download cif 
    cif_url = f'https://files.rcsb.org/view/{pdbid.upper()}.cif'
    cif_path = os.path.join(download_dir,f'{pdbid.upper()}.cif')
    response = requests.get(cif_url)
    with open(cif_path,'wb') as f:
        f.write(response.content)
    # Download fasta
    fasta_url = f'https://www.rcsb.org/fasta/entry/{pdbid.upper()}'
    fasta_path = os.path.join(download_dir,f'{pdbid.upper()}.fasta')
    response = requests.get(fasta_url)
    with open(fasta_path,'wb') as f:
        f.write(response.content)                         
    return cif_path, fasta_path

#### Step 2 - Download Relevant PDBIDs 

In [4]:
pdbid = '1CRN'
cif_file, fasta_file = download_pdb(pdbid)
print(cif_file)
print(fasta_file)

Data/pdb_1CRN/1CRN.cif
Data/pdb_1CRN/1CRN.fasta


In [5]:
#execute this code cell to download and view files
pdbid = '2A3N'
cif_file, fasta_file = download_pdb(pdbid)
print(cif_file)
print(fasta_file)

Data/pdb_2A3N/2A3N.cif
Data/pdb_2A3N/2A3N.fasta


#### Step 3 - Using Phenix AutoSol to solve structure for 2A3N
We will go through the steps and provide screenshots for solving the structure of `2A3N`. Once you complete the activity for `2A3N`, feel free to repeat the process with `1CRN`.

**Step 3.1** - Open Phenix, click on *Experimental Phasing*, and choose <u>AutoSol</u>:
<center><img src='images/submod_2/submod2_image1.png'
     align='middle'
     width='600'/> <br></center>

**Step 3.2** - This will open a separate AutoSol window. <u>Click</u> on the Browse button to access the folder containing your structure factor files. 
<center><img src='images/submod_2/submod2_image2.png'
     align='middle'
     width='600'/> <br></center>
     
- *Note: In this example we are using a crystallographic information file (.cif), as shown in the image below, but this could also be a .sca file generated by Scalepack after scaling data collected at an x-ray source.*
<center><img src='images/submod_2/submod2_image3.png'
     align='middle'
     width='600'/> <br></center>

**Step 3.3** - AutoSol will automatically recognize the reflection labels, wavelength, and unit cell parameters. All of these terms were defined during scaling. The anomalous scattering atom must be indicated. This dataset was collected with selenomethionine derivatized protein, so we <u>must indicate that the anomalous atom is Se</u>. If we know how many Se atoms are expected in the unit cell we can add that too, but this line may also be left blank since the next step will be to include the sequence file. 
<center><img src='images/submod_2/submod2_image4.png'
     align='middle'
     width='600'/></center>

**Step 3.4** - Click *Browse* again and select the FASTA file containing the protein sequence. AutoSol is now ready to be run by clicking the *Run* button as shown below. Once the job is complete there will be a list of files generated.
<center><img src='images/submod_2/submod2_image5.png'
     align='middle'
     width='600'/> <br></center>

*Note: The NCS copies field refers to noncrystallographic symmetry which is present whenever data are collected from a crystal containing two or more copies of a molecule not related by the normal symmetry of the crystal.*

**Step 3.5** - Click the *Open in Coot* button to open the crystallographic object-oriented toolkit (Coot) which is a molecular graphics program used for visualizing and building protein structures. Also, there are statistical values associated with the refinement of the protein structure called R-work and R-free. R values are basically a way of calculating the agreement between x-ray data and a model that has been generated from it. Values range from 0 - 0.6, with any value over 0.5 being unusable. Fully refined proteins typically have R values below 0.3 depending on the resolution and data quality. In this example AutoSol built much of the protein on its own. It did a pretty good job doing so as indicated by the low R values.
<center><img src='images/submod_2/submod2_image6.png'
     align='middle'
     width='600'/> <br></center>


Opening the structure in COOT (as shown in the image below) reveals the protein structure built by AutoSol as well as its electron density map.
<center><img src='images/submod_2/submod2_image7.png'
     align='middle'
     width='600'/> </center>

**Step 3.6** - *Zoom in* to observe the details of this 1.35 Å map. The phenylalanine ring shows a hole in the center and the remaining backbone atoms are clearly outlined by the blue electron density map. The red arrows show the location of one the selenium atom in one of the selenomethionine residues (labeled MSE in Coot). There are two conformations of this residue, so two Se atoms are shown.
<center><img src='images/submod_2/submod2_image8.png'
     align='middle'
     width='600'/> <br></center>

### Key Takeaways
Most of the residues in this structure were automatically built with the exception of a few on the N-terminus. Lower resolution and lower quality data will have gaps in the model that have to be filled in manually. 

-------
## Activity 2 - Solving a Structure Using Molecular Replacement

### Requirements
* A structure factor file from the data set of the protein you want to solve.
* A .pdf file of your search model that you believe is similar in structure to the protein you want to solve.
* A .fasta file containing the search model’s sequence.

### Solving Strucuture Using Phenix Phaser-MR

#### Step 1 - Identify protein with similar structure
**Step 1.1** - The first step is to find a search model required to solve the structure using the method of molecule replacement. The model chosen should be one that is believed to be similar in structure to the protein we want to solve. We determine this by looking at proteins that already have solved structures from PDB with high sequence identity to our new unsolved protein. This is done using the *Basic Local Alignment Tool (BLAST)* from the National Center for Biotechnology Information (NCBI) website. All that is required is a protein sequence. Let’s use a protein sequence for glyceraldehyde 3-phosphate dehydrogenase (GAPDH) from humans taken from the NCBI website:
<center><img src='images/submod_2/submod2_image9.png'
     align='middle'
     width='600'/> <br></center>
     
**Step 1.2** - [Navigate to the BLAST website](https://blast.ncbi.nlm.nih.gov/Blast.cgi) and choose Protein BLAST to search the database for suitable search models.

<center><img src='images/submod_2/submod2_image10.png'
     align='middle'
     width='600'/></center>
     
**Step 1.3** - Paste the FASTA sequence into the sequence window and then set the Database to PDB proteins. This will limit the search to proteins that have known structures, which is what we are looking for in a search model for molecular replacement. 
<center><img src='images/submod_2/submod2_image11.png'
     align='middle'
     width='600'/></center>

#### Step 4 - Choose optimal hit.
Hits will be ordered from best to worst. The list below shows several options for molecular replacement, but typically we choose search models from the list that are as similar to our protein as possible. The top hit from the list below is from Sus scrofa (wild boar). With a percent identity of 95.78% it is certain that this will be a suitable search model. However, farther down the list are search models that should also be suitable in the 72-75% range. However, since we have one option that is very close we should choose that one. The high identity will mean there are fewer residues that will have to be changed during model building and refinement.
<center><img src='images/submod_2/submod2_image12.png'
     align='middle'
     width='600'/></center>

#### Step 5 - Download PDB file.
The column furthest to the left shows the PDB accession code, so from here we can get all the remaining files necessary from the PDB. [Navigate to the PDB website](https://www.rcsb.org/) and enter `5TSO` into the search bar. Then download the .pdb file to be used for the search model by clicking the *Download Files* button and choosing *PDB Format*.

#### Step 6 - Setup molecular replacement analysis in Phenix
Open Phenix, click on Molecular Replacement, and choose Phaser-MR (simple interface).
<center><img src='images/submod_2/submod2_image13.png'
     align='middle'
     width='600'/> <br></center>


*Note: This will open a separate Phaser window as shown below:*
<center><img src='images/submod_2/submod2_image14.png'
     align='middle'
     width='600'/> <br></center>
     
#### Step 7 - Choose relevant data files.
Click on *Add file* and choose the data file of the structure you want to solve, the .pdf file of your search model, and a .fasta file of the search model’s sequence.
<center><img src='images/submod_2/submod2_image15.png'
     align='middle'
     width='600'/> </center>

#### Step 8 - Manually enter required parameters.
As with AutoSol, each file is correctly recognized, and the unit cell parameters are automatically entered, but two additional fields must be filled out before running the job. <br>

**Step 8.1** - First, include the sequence identity that was reported in the BLAST search. <br>
**Step 8.2** - Second, include the number of polypeptide copies that are found in the search model. To do this you have to look at the .pdb file of the search model. For this particular search model there are three copies of GAPDH, so we have to indicate that in the Component copies field. <br>
**Step 8.3** - Running the job will give several outputs, but we can start by looking at the model and electron density map in COOT.
<center><img src='images/submod_2/submod2_image16.png'
     align='middle'
     width='600'/></center>

#### Step 9 - Analyze electron density map.
The resulting electron density map shows clear density for an alpha helix as well as a ligand binding site. From here it is necessary to look at each residue to adjust or change them to match the new protein structure to remove model bias from the search model.
<center><img src='images/submod_2/submod2_image17.png'
     align='middle'
     width='600'/></center>
     
     

----------
## 📖 Submodule 2 - Test Your Knowledge

In [1]:
#Display quiz questions
from IPython.display import IFrame
IFrame('quiz_files/submod2/quiz2_mc.html', width=1000, height=600)

In [ ]:
#Display the quiz and select the correct answers from the dropdown menu for each question
from quiz_files.quiz_module import run_quiz
run_quiz(import_type="json", import_path="quiz_files/dropdown_quiz.json", instant_feedback=False, shuffle_questions=False, shuffle_answers=True)

## Conclusion
In Submodule 2 we explored the pivotal processes involved in solving macromolecular structures using diffraction data. Starting from scaled reflections, we advanced through techniques such as molecular replacement and anomalous diffraction methods (SAD/MAD), emphasizing the practical application of these techniques through the Phenix software suite. With its user-friendly interface and comprehensive tools, Phenix provides user-friendly tools for structure determination in macromolecular crystallography. In module 3 we will take the solved structure and perform model building and refinement in COOT.

## Clean Up
<div class="alert alert-block alert-warning"> <b>Attention:</b> Remember to shutdown VM and delete any relevant resources</a>. </div>